# Lottery Demo

### By: Duke Alf
### Date: April 2021

## Decentralized Lottery App on the Ethereum Blockchain

### Overview:
This is a demo for a simple lottery Dapp. The traditional lottery system is inherently unfair to players. Winnings are lost due to splitting pots, paying taxes, and fraud. This app allows players to join a lottery for the cost of 1 ether. Each address can only enter the lottery once. The winner is randomly selected and recieves the entire payout. 

### Process:
-the owner of the contract opens the lottery 
-players can enter the lottery by sending 1 ether
-the ether is pooled together
-the owner closes the lottery and calls a function to randomly select a winner
-the winner and payout is visible to all
-the winning address recieves the payout
-the owner can reset the lottery and the process can begin again

### Contracts:
There is one Solidity contract called Lottery.sol. This contains all necessary information to successfully conduct the lottery. This can be found in the contracts folder. 

### Demo:
Run the cells in order and follow comments. The blockchain I use to deploy and test the contract is [eth-tester](https://github.com/ethereum/eth-tester). This supports the PyEVM backend, which is a full Ethereum implementation in Python. I interact with PyEVM using Web3. Eth-tester will automatically be set to interact with PyEVM.

`pip install web3[tester]` installs `Web3.py`, `eth-tester`, and `PyEVM`

In [158]:
from solcx import set_solc_version, compile_files
from web3 import Web3

from eth_tester import EthereumTester, PyEVMBackend

import os

In [159]:
set_solc_version('v0.7.6')

In [160]:
# EthereumTester used explicitly to get accounts
TESTER = EthereumTester(backend=PyEVMBackend())

The following functions are used to compile contracts. 

In [161]:
def get_contract_path(contract_name):
    '''
    construct path to contract file
    assumes there is a subdirectory in your current working directory named "contracts"
    
    contract_name: the name of the contract (without the .sol suffix)
    '''
    return os.path.join(os.getcwd(), 'contracts', f'{contract_name}.sol')

def compile_contract(contract_name):
    '''
    compile contract and get the result of compilation
    
    contract_name: the name of the contract, which should match the filename (without the .sol suffix)
    '''
    source_file_name = get_contract_path(contract_name)
    compiled_sol = compile_files([source_file_name]) # Compiled source code
    return compiled_sol[source_file_name + ":" + contract_name]

`get_w3` initiates a web3 object which we use to interact with the backend

In [163]:
def get_w3(provider, account=None):
    '''
    get a web3 object instance
    sets the default account to the one provided or the first test account if not provided
    
    provider: backend test provider
    account:  the account number to set as default, or None if you want to use the first test account
    '''
    # web3.py instance
    w3 = Web3(provider)
    
    if account:
        # set account as default sender of transactions
        w3.eth.defaultAccount = account
    else:
        # set the first test account as default sender of transactions
        w3.eth.defaultAccount = w3.eth.accounts[0]
    
    return w3

Now, we deploy the contract to the blockchain. We pass in the `w3` object, `compiled_contract`, and `c_args` (which is any arguments taken in the constructor of the contract - optional).

We make a transaction with the blockchain by deploying our contract.

Once the transaction is complete, we must return its address, abi, and bytecode through `deployed_contract_interface`. We also return `tx_receipt` which holds information about the transaction such as index, gas used, block number, address, etc. 

In [164]:
def deploy_contract(w3, compiled_contract, *c_args):
    '''
    deploy the contract
    
    w3: web3 object instance
    compiled_contract: dictionary of compiled contract should have values for the keys: "abi" and "bin"
    *c_args: a variable number of contract arguments to pass to the constructor of the contract
    '''
    # get the contract interface from the compiled contract
    contract_interface = w3.eth.contract(abi=compiled_contract['abi'], bytecode=compiled_contract['bin'])
    
    # Instantiate the contract by passing in constructor args, and submit the transaction to deploy
    tx_hash = contract_interface.constructor(*c_args).transact()
    
    # Wait for the transaction to be put into a block, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Get the deployed contract instance from the blockchain with the newly-deployed contract's address
    deployed_contract_interface = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=compiled_contract['abi'],
        bytecode=compiled_contract['bin']
    )
    
    return tx_receipt, deployed_contract_interface

These are demo accounts we will use to test the contract. 

In [165]:
TESTER.get_accounts()

('0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528')

In [166]:
owner = TESTER.get_accounts()[0]
alice = TESTER.get_accounts()[1]
bob = TESTER.get_accounts()[2]
joe = TESTER.get_accounts()[3]
mark = TESTER.get_accounts()[4]
sara = TESTER.get_accounts()[5]

Now, we compile the contract and connect to the blockchain. 

In [167]:
sc_compiled = compile_contract("Lottery")
provider = Web3.EthereumTesterProvider(ethereum_tester=TESTER)

Using the provider, we get the web3 object, passing in the default account which is the owner of the lottery. The returned `Web3` object, connected to the `PyEVM` backend through `eth-tester`, is how we interact with the blockchain.

In [168]:
sc_w3 = get_w3(provider, owner)

We deploy to the blockchain using the above and get back our receipt and contract interface/information. 

In [169]:
sc_receipt, sc_contract = deploy_contract(sc_w3, sc_compiled)

Here are the functions we can call. 

In [170]:
sc_contract.all_functions()

[<Function closeLottery()>,
 <Function generateWinner()>,
 <Function joinLottery()>,
 <Function num_players()>,
 <Function openLottery()>,
 <Function playerList(uint256)>,
 <Function resetLottery()>,
 <Function winner()>]

The following functions make it easier to execute transactions. 

In [171]:
def exec_call(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a call, which does not execute a transaction (i.e. no write)
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)

    return_value = func_inst(*f_args).call(transaction)
    return return_value

def exec_transact_receipt(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a transaction (i.e. a write), and return the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)
    
    # get the return value first, without executing transaction
    return_value = exec_call(contract_interface, function_name, *f_args, transaction=transaction)
    
    # execute the transaction
    tx_hash = func_inst(*f_args).transact(transaction)
    # receipt does not contain values returned by function
    tx_receipt = contract_interface.web3.eth.waitForTransactionReceipt(tx_hash)
    
    return return_value, tx_receipt

def exec_transact(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute transaction, but ignore the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    rv, _ = exec_transact_receipt(contract_interface, function_name, *f_args, transaction=transaction)
    return rv

Now, we can start interacting with the Lottery. 

To start, I will show that you can't join the lottery until the owner opens it, and only the owner can do this. 

Let's try to have alice join the lottery before it is open:

In [172]:
try:
    exec_transact(sc_contract, "joinLottery", transaction={'from': alice,'value': sc_w3.toWei(1, 'ether')})
except:
    print("Lottery is not open yet. Check back later.")

Lottery is not open yet. Check back later.


Let's have alice try to open the lottery:

In [173]:
try:
    exec_transact(sc_contract, "openLottery", transaction={'from': alice})
except:
    print("Only the Owner can perform this action.")

Only the Owner can perform this action.


Now, we will demonstrate the owner opening the lottery and addresses properly buying into the lottery for 1 ether. 

In [174]:
# owner opens lottery
openID, open_receipt = exec_transact_receipt(sc_contract, "openLottery", transaction={'from': owner})

In [175]:
openID

[]

In [176]:
open_receipt

AttributeDict({'transactionHash': HexBytes('0xdc5b6aed4503befbd5cb491460b60431614d6fbd734a452b85466dde8f47aec5'),
 'transactionIndex': 0,
 'blockNumber': 2,
 'blockHash': HexBytes('0x5096db7525e122a2f1aec9ca930f4f66f81e2a1c32d39b7f87dd7633ec47396e'),
 'cumulativeGasUsed': 42942,
 'gasUsed': 42942,
 'contractAddress': None,
 'logs': [],
 'status': 1})

In [177]:
# others join the lottery
exec_transact(sc_contract, "joinLottery", transaction={'from': alice,'value': sc_w3.toWei(1, 'ether')})
exec_transact(sc_contract, "joinLottery", transaction={'from': bob,'value': sc_w3.toWei(1, 'ether')})
exec_transact(sc_contract, "joinLottery", transaction={'from': joe,'value': sc_w3.toWei(1, 'ether')})
exec_transact(sc_contract, "joinLottery", transaction={'from': mark,'value': sc_w3.toWei(1, 'ether')})
exec_transact(sc_contract, "joinLottery", transaction={'from': sara,'value': sc_w3.toWei(1, 'ether')})

[]

In [178]:
# anyone can see how many people are in the lottery
exec_call(sc_contract, "num_players")

5

In [179]:
# anyone can query for the winner, which has not been chosen yet
exec_call(sc_contract, "winner")

'0x0000000000000000000000000000000000000000'

In [180]:
# the addresses currently in the lottery can be queried
exec_call(sc_contract, "playerList",2)

'0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718'

In [181]:
# owner tries to pick winner before the lottery closes
try:
    exec_transact(sc_contract, "generateWinner", transaction={'from': owner})
except:
    print("Lottery must be closed to declare a winner.")

Lottery must be closed to declare a winner.


In [182]:
# owner closes lottery
exec_transact(sc_contract, "closeLottery", transaction={'from': owner})
# see the balance of the lottery
sc_w3.eth.getBalance(sc_receipt.contractAddress)

5000000000000000000

In [183]:
# owner randomly generates winner
exec_transact(sc_contract, "generateWinner", transaction={'from': owner})

[]

In [184]:
# the winner is credited
winner_address = exec_transact(sc_contract, "winner", transaction={'from': owner})
TESTER.get_balance(winner_address)

1000003999999999999939961

In [185]:
# loser gets nothing
loser_address = exec_call(sc_contract, "playerList",2)
TESTER.get_balance(loser_address)

999998999999999999937343

In [186]:
# see the balance of the contract is now 0
sc_w3.eth.getBalance(sc_receipt.contractAddress)

0

In [187]:
# owner resets lottery and the process can begin again
exec_transact(sc_contract, "resetLottery", transaction={'from': owner})

[]

In [188]:
# now, there is no winner
exec_transact(sc_contract, "winner", transaction={'from': owner})

'0x0000000000000000000000000000000000000000'

In [189]:
# there are no players
exec_transact(sc_contract, "num_players", transaction={'from': owner})

0

## Resources and Notes

### How to address randomness:
100% true randomness in Ethereum is almost impossible to achieve since everything is deterministic. The most effective solutions are to use oracles or generate random numbers off chain. A solution such as Chainlink VRF (Verifiable Random Function) is provable random, but I decided not to pursue this because I would need to send LINK to generate a number this way. Instead, I have a solution that could potentially be exploited by the miner, but it would be pretty difficult. The owner closes the lottery so no one else can enter. Then, the owner calls a function to generate a winner. The random number is generated from the hash of the block.timestamp + block.difficulty. Since the players do not know when the owner will call the function to choose a winner and the players cannot predict the exact timestamps of future blocks, this solution should be mostly fair. Potential problems come from miners manipulating the timestamp and difficulty to get a hash they are looking for. But if they are waiting for a specific time or difficulty, someone else will hash the block before them. Overall, I think this is a solid method for this scenario, but certainly should not be used on the real blockchain.

References:
https://ethereum.stackexchange.com/questions/191/how-can-i-securely-generate-a-random-number-in-my-smart-contract

https://stackoverflow.com/questions/58188832/solidity-generate-unpredictable-random-number-that-does-not-depend-on-input

https://www.sitepoint.com/solidity-pitfalls-random-number-generation-for-ethereum/

https://blog.chain.link/random-number-generation-solidity/

### Challenges:
Initially, I hoped to deploy the contract to the Rinkby testnet using Infura, but I would have had to create numerous Metamask accounts in order to interact with the lottery contract. I also wanted to build a "Can't-Lose Lottery" where the pooled lottery money is used to generate interest which is sent to a winner and everyone else gets their initial investment back. However, I could not efficiently simulate the process of generating interest on the pooled ether. 

### Sources:
I want to thank Nodari Gogoberidze for being a great lecturer for CSCI 118 and teaching me about the blockchain. The functions that I incorporated that involved compiling and running transactions were based his those used in his demos throughout the course. 

eth-tester: https://github.com/ethereum/eth-tester

solidity documentation: https://docs.soliditylang.org/en/v0.8.4/
